In [1]:
__author__ = "dataviral"

# Jaccard Similarity using Locality Sensitive Hashing

- Many problems in NLP require the calculation jaccard similarity of long sequences of text
- The datasketch library provides an implementation of the Locality Sensitive Hashing algorithm which makes computing jaccard similarity very fast
- This notebook first describes how to use datasketh to infer senteces with high jaccard similarity from a medium size movie review dataset.
- The Last section describes shows a naive implementation for jaccard similarity.
- Comparing the 2 approaches; the efficiency of LSH for computing similarity between sentences beomes evident.

*PS: Jaccard similarity is an indicator of **syntactic similarity***

### Install requires libs

In [ ]:
!pip install datasketch
!pip install pandas
!pip install tqdm

In [17]:
import pandas as pd
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm

### Load Data

In [4]:
DATA_PATH = "./data/imdb/imdb_master.csv" # https://www.kaggle.com/utathya/imdb-review-dataset
df = pd.read_csv(DATA_PATH, encoding='latin1', index_col=0)

### Store data as key value pairs

In [13]:
reviews = dict()
for key, value in enumerate(df.review.values):
    reviews[key] = value

### Create LSH object for jaccard calculations

In [14]:
lsh = MinHashLSH(
    threshold= 0.5, # Jaccard similarity threshold
    num_perm=128,   # Number of hash functions
)

### Update the Hash

In [18]:
for key in tqdm(reviews):
    review_hash = MinHash()
    # Iterate over words and add to the review specific hash
    for word in reviews[key].split(" "):
        review_hash.update(word.encode("utf-8"))
    
    # Now insert the review specific hash to lsh object
    lsh.insert(key, review_hash)

100%|██████████| 100000/100000 [06:07<00:00, 271.85it/s]


### Query from the hash

In [23]:
def query(lsh, sentence):
    """ builds the hash for the sentence and query's the hash for similar senteces """
    sentence_hash = MinHash()
    for word in sentence.split(" "):
        sentence_hash.update(word.encode("utf-8"))
    
    similar_sentences_ids = lsh.query(sentence_hash)
    return similar_sentences_ids

In [49]:
# Pick a sentence in the dataset to use as query senteces
query_review_id = 200

query_review = reviews[query_review_id] # movie review with key as 1000
print("Query Sentence: \n", query_review)

Query Sentence: 
 This is the third movie in a month I have watched that did not go the way I expected. The first two being The Black Dahlia and Hollywoodland, neither of which gave any new ideas of who committed the crimes.<br /><br />I have always had a fascination with UFOs and was so excited to see a new movie on the subject of UFO investigation that was not a comedy. But after about 30 minutes, it all went horribly wrong.<br /><br />I could have stood for the acting, the camera angles, the stereotypes if only there was a good story about chasing UFOs, but none here. I am not saying there was anything wrong with the subject matter, but Netflix pushed this movie as a UFO skeptic and a UFO believer investigating multiple sitings.<br /><br />I stopped watching about half way thru. Can't believe I wasted that much time with this one. Please don't make the same mistake I did.


In [50]:
# Find all sentences in the dataset similar to the query sentence
similar_reviews_ids = query(lsh, query_review)

In [52]:
# Show the similar sentences
for i, review_id in enumerate(similar_reviews_ids):
    if review_id == query_review_id: continue
    print("Similar review {}:\n".format(i))
    print(reviews[review_id])
    print("---------\n")

Similar review 1:

Wow. I went to the video store tonight because I was in the mood for a bad B Horror movie and I found this Gem. I looked at the cover and I thought it looked like just the movie for my mood. I brought it home and put it on.<br /><br />This movie was not the B Horror movie that I had in mind. This was MUCH worse. I wanted a bad movie but what I got, I didn't know that crap like this existed amongst man. This movie seemed like a 5 year old wrote and directed it and that is being nice about it.<br /><br />I am an aspiring director and this movie made me so mad that someone out there is actually paying this guy to direct movies. He needs to work at a garbage dump shoveling crap where he belongs.<br /><br />If you are thinking about renting this or buying it. I will tell you the same thing that I would tell someone getting ready to commit suicide. "DON'T DO IT, IT'S NOT WORTH IT!" I really have nothing nice to say about this movie. DON'T DO IT!
---------

Similar review 2

# The Naive Way

### Function to calculate jaccard similarity b/w 2 sentences the naive python way 

In [54]:
def compute_jaccard(sentence1, sentence2):
    words_in_sentence1 = set(sentence1.split(" "))
    words_in_sentence2 = set(sentence2.split(" "))
    
    return len(words_in_sentence1 & words_in_sentence2) / len(words_in_sentence1 | words_in_sentence2)

### Finding similar senteces in the dataset using the naive python way.

In [70]:
def get_similar_senteces_naive(sentence, reviews, jaccard_thresh=0.5):
    similar_sentences_ids = []
    for key in reviews:
        js = compute_jaccard(sentence, reviews[key])
        if js >= jaccard_thresh:
            similar_sentences_ids.append(key)
    return similar_sentences_ids

In [63]:
# Naive method
%timeit get_similar_senteces_naive(query_review, reviews, 0.5)

5.69 s ± 175 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
# Using lsh
%timeit query(lsh, query_review)

2.62 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# The speed up

In [69]:
print("LSH is about {} times faster".format((5.69 * 1000) / 2.62))

LSH is about 2171.7557251908397 times faster
